In [1]:
#Speech to Text Cell 2 (Continous Speech Recognition, with continous language detection #not natural needs to pause for 1 sec when code switching
#Link: https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/language-identification?tabs=once&pivots=programming-language-python

import azure.cognitiveservices.speech as speechsdk
import json
import time
import math


def speech_recognize_continuous_from_file2(filePath): 
    #Stop Callback
    def stop_cb(evt):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        #print('CLOSING on {}'.format(evt))
        nonlocal done
        done = True

    #Handle Recognized Speech Envent


    
    endpoint_string = "wss://{}.stt.speech.microsoft.com/speech/universal/v2".format("southafricanorth")

    speech_config = speechsdk.SpeechConfig(subscription="0915a529a519455daa5e8dfce0a921df", endpoint=endpoint_string)
    speech_config.set_property(property_id=speechsdk.PropertyId.SpeechServiceConnection_ContinuousLanguageIdPriority, value='Latency')
    speech_config.set_service_property("wordLevelConfidence","true", speechsdk.ServicePropertyChannel.UriQueryParameter)
    speech_config.request_word_level_timestamps()      #Gets offset & duration of each word                                             
    speech_config.set_profanity(speechsdk.ProfanityOption.Removed)  #Removes profane words
    speech_config.set_service_property("format","detailed", speechsdk.ServicePropertyChannel.UriQueryParameter)

    audio_config = speechsdk.audio.AudioConfig(filename=filePath)                                                         #path of file here
    
    auto_detect_source_language_config = speechsdk.languageconfig.AutoDetectSourceLanguageConfig(languages=["en-US", "ar-EG"])
    
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, auto_detect_source_language_config=auto_detect_source_language_config, audio_config=audio_config)


    #Stop Callback
    done = False

    stringList=[]
    
    def handleResult(evt):          #handles result of every small output
        r= evt.result
        #print(r.text)
        #output= json.loads(r.json)['NBest'][0]            # the display text of the result is always associated with ['NBest'][0]       
        stringList.append(r.text)
        
    

    #Recognized Speech
    speech_recognizer.recognized.connect(lambda evt: handleResult(evt) )  # only occurs if speech is recognized (r.reason == speechsdk.ResultReason.RecognizedSpeech) (so you don't need to handle the speech not recognized condition

    # Stopping Conditions (of continous speech recognition-> either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()


    #Delay between code
    while not done:
        time.sleep(.5)

    # Stop continuous speech recognition & Return output
    speech_recognizer.stop_continuous_recognition()

    string=""
    for x in stringList:
        string+= x + " "
        
    print(string)    
    return string



In [17]:
file= '2-3'
path= "Sample Audio/Code-switched/" + file +".wav"

x= speech_recognize_continuous_from_file2(path)

I'm going to the park. مش عارف أعمل إيه دلوقتي؟ 
